<a href="https://colab.research.google.com/github/Kim-JungHyun01/AIStudy24/blob/master/%ED%85%8C%EC%8A%A4%ED%8A%B8/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검증테스트

- 일반적인 알고리즘 : 훈련세트 & 테스트세트 분류
- 테스트세트로 평가를 하면 테스트세트에 맞는 모델이 만들어짐

    => 과소,과대적합만 안될려고 하기에 <검증>이 필요
- 테스트 세트에 일반화 성능을 올바르게 예측하기 위해서는 가능한 테스트세트를 사용하면 X
    
    => 모델을 만들고 마지막에 딱 한번 사용하는 것이 좋음
- 결정트리의 max_depth 매개변수를 사용한 하이퍼파라미터 튜닝진행
<br/><br/>
- 테스트 세트를 사용하지 않으려면 모델이 과소, 과대적합인지 판단이 어려움
- 테스트 세트를 사용하지 않고 측정하는 방법 : 훈련세트를 또 나누는 것
    - 훈련세트 : 검증세트 : 테스트세트 = 60 : 20 : 20
    - 진행
        1. 테스트하고 싶은 매개변수를 변경(가장 좋은 모델이 나올 수 있도록 하는 것)
        2. 매개변수를 사용해 훈련세트, 검즘세트를 합침
        3. 전체 훈련 데이터에서 모델을 다시 훈련함(훈련 : 훈련세트, 평가 : 검증세트)
        4. 마지막 테스테에서 최종 점수 평가(테스트세트)

# 데이터 input, target
- train : sub(훈련용), val(검증용)
- test : 테스트용
<br/><br/>
- total(100%) => train : test = 80% : 20%
- train(80%) => sub: val = 60% : 20%

In [2]:
import pandas as pd

wine = pd.read_csv('http://bit.ly/wine_csv_data')

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

#tarin, test = 80 : 20 => test_size = 0.2
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

#sub : 검증(val) = 60 : 20
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

print("train : ", train_input.shape, " test : ", test_input.shape)
print("sub : ",sub_input.shape,"val : ",val_input.shape)

train :  (5197, 3)  test :  (1300, 3)
sub :  (4157, 3) val :  (1040, 3)


# 결정트리 훈련(sub)

In [5]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print("sub : ", dt.score(sub_input, sub_target))
print("val : ", dt.score(val_input, val_target))
# 결론 : tarin(sub)(0.997) : train점수 100%에 가까움(과대적합)
# 매개변수 조정필요

sub :  0.9971133028626413
val :  0.864423076923077


# 교차검증(cross_validate)
- 검증세트생성으로 인해 훈련세트의 양이 줄어듬(많은 데이터를 훈련하면 좋은 결과가 나올 가능성이 높음)
- 교차검증 진행
    - 훈련, 검증세트를 나누어 평가하는 방식을 여러번 반복하는 것
    - ex) 3-폴드, 5-폴드(기본), 10-폴드
- 결과
    - (k,v)의 딕셔너리형태로 반환
        1. fit_time(훈련시간)
        2. score_time(검증시간)
        3. test_score(최종점수) : 검증폴드의 점수
    - 최종결과는 '평균(mean)'으로 할 것

- 주의
    - 훈련세트를 섞어서 폴드를 나누지 않음
        - tarin_test_split()함수로 전체 데이터를 섞은 후 훈련세트 준비
        - 만약 교차검증을 할때 섞을 경우 분할기(splitter)를 지정
    - 폴드를 어떻게 나눌 것인지 기준설정
    - 기본적으로 회귀모델인 KFold 분할기를 사용
    - 분류 모델인 경우,
        target 클래스를 골고루 나누기 위해 StratifiedKFold 이용

<img src = "https://velog.velcdn.com/images%2Fheidi%2Fpost%2Fd9921883-ad1d-4855-b962-d4f162d91507%2Fimage.png" width = 450px>

In [6]:
from sklearn.model_selection import cross_validate

# 평가할 모델 객체를 첫번째 매개변수로 전달(직접 검증세트를 구분하지 않고 훈련세트 전체를 전달)
# dt : 결정트리 알고리즘 객체
scores = cross_validate(dt, train_input, train_target, cv = 5)
# cv : 폴드의 수(기본값 : 5)_반복횟수

print(scores)

"""
{'fit_time': array([0.00916362, 0.00838637, 0.00910163, 0.00918841, 0.00849676]),
'score_time': array([0.00124025, 0.00130463, 0.00137353, 0.00137806, 0.00139999]),
'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
"""

{'fit_time': array([0.00916362, 0.00838637, 0.00910163, 0.00918841, 0.00849676]), 'score_time': array([0.00124025, 0.00130463, 0.00137353, 0.00137806, 0.00139999]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [12]:
import numpy as np

print("성능평가(평균) : ",np.mean(scores['test_score']))
print("test           : ", dt.score(test_input, test_target))
# 결론 : train(0.855) < test(0.857) : 과소적합

성능평가(평균) :  0.855300214703487
test           :  0.8569230769230769


## StratifiedKFold
- 교차검증시 데이터 분할기
- train_test_split()을 사용하지 않고 바로 셔플 및 분할을 진행할때 사용

In [13]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print("성능평가(평균) : ",np.mean(scores['test_score']))
print("test           : ", dt.score(test_input, test_target))
# 결론 : train(0.855) < test(0.857) : 과소적합

성능평가(평균) :  0.855300214703487
test           :  0.8569230769230769


In [14]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)#10-폴드 교차검증수행
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print("성능평가(평균) : ",np.mean(scores['test_score']))
print("test           : ", dt.score(test_input, test_target))
# 결론 : train(0.8574) > test(0.8569) : 적당

성능평가(평균) :  0.8574181117533719
test           :  0.8569230769230769
